#### nDCG

nDCG (Normalized Discounted Cumulative Gain) – популярная метрика в задаче ранжирования, которая учитывает порядок элементов в выдаче. Мы поэтапно разберёмся, что означает каждая буква в данной аббревиатуре, одну за другой реализовав метрики CG, DCG и nDCG.  

In [1]:
import warnings
warnings.filterwarnings('ignore')

#### Cumulative Gaine
Что бы понять, какая модель ранжирует лучше, давайте для двух версий приложения сложим численные оценки релевантности, которые мы ввели ранее.

In [16]:
from typing import List

import numpy as np


def cumulative_gain(relevance: List[float], k: int) -> float:
    """Score is cumulative gain at k (CG@k)

    Parameters
    ----------
    relevance:  `List[float]`
        Relevance labels (Ranks)
    k : `int`
        Number of elements to be counted

    Returns
    -------
    score : float
    """
    #top_relevance = sorted(relevance, reverse=True)
    score = np.sum(relevance[:k])
    return score

In [17]:
relevance = [0.99, 0.94, 0.88, 0.74, 0.71, 0.68]
k = 5
print(cumulative_gain(relevance, k))

4.26


#### Discounted Cumulative Gain

Функция для подсчёта DCG.

DCG@k=∑i=1klog2(i+1)reli

In [ ]:

from typing import List

import numpy as np


def discounted_cumulative_gain(relevance: List[float], k: int, method: str = "standard") -> float:
    """Discounted Cumulative Gain

    Parameters
    ----------
    relevance : `List[float]`
        Video relevance list
    k : `int`
        Count relevance to compute
    method : `str`, optional
        Metric implementation method, takes the values \
            `standard` and `industry`

    Returns
    -------
    score : `float`
        Metric score
    """    
    score = 0
    if method == "industry":
        for i, v in enumerate(relevance[:k], 1):
            score += (2*v-1)/np.log2(i+1)
            return score
    for i, v in enumerate(relevance[:k], 1):
        score += v/np.log2(i+1)
    return score

In [23]:
relevance = [0.99, 0.94, 0.88, 0.74, 0.71, 0.68]
k = 5
method = 'standard'
print(discounted_cumulative_gain(relevance, k, method))

2.6164401144680056


#### Normalized Discounted Cumulative Gain

DCG@k - не нормированная метрика, что усложняют задачу сравнения. Один из простых методов нормирования — поделить на максимальное число. Что если мы посчитаем уже известную нам DCG, но изначально отсортируем релевантности в порядке убывания. Мы получим максимальное значение DCG для конкретного запроса и заданного набора выдачи (параметр k) или IDCG (Ideal Discounted Cumulaive Gain). 

Теперь, что бы получить nDCG (Normalized Discounted Cumulative Gain) достаточно поделить DCG на IDCG, а метрика станет нормированной от 0 до 1.  

In [24]:
from typing import List

import numpy as np


def normalized_dcg(relevance: List[float], k: int, method: str = "standard") -> float:
    """Normalized Discounted Cumulative Gain.

    Parameters
    ----------
    relevance : `List[float]`
        Video relevance list
    k : `int`
        Count relevance to compute
    method : `str`, optional
        Metric implementation method,
        takes the values `standard` and `industry`

    Returns
    -------
    score : `float`
        Metric score
    """
    dcg = 0
    idcg = 0
    if method == "industry":
        for i, v in enumerate(relevance[:k], 1):
            dcg += (2*v-1)/np.log2(i+1)
            
        for i, v in enumerate(sorted(relevance[:k], reverse = True), 1):
            idcg += (2*v-1)/np.log2(i+1)

    else:   
        for i, v in enumerate(relevance[:k], 1):
            dcg += v/np.log2(i+1)  

        for i, v in enumerate(sorted(relevance[:k], reverse = True), 1):
            idcg += v/np.log2(i+1)
    score = dcg/idcg
    return score

In [25]:
relevance = [0.99, 0.94, 0.74, 0.88, 0.71, 0.68]
k = 5 
method = 'standard'
print(normalized_dcg(relevance, k, method))

0.9962906539247512


#### Average Normalized Discounted Cumulative Gain
Avarage nDCG - усредненное значение метрики nDCG по каждому запросу из множества. 

In [36]:
from typing import List

import numpy as np

def avg_ndcg(list_relevances: List[List[float]], k: int, method: str = 'standard') -> float:
    """avarage nDCG

    Parameters
    ----------
    list_relevances : `List[List[float]]`
        Video relevance matrix for various queries
    k : `int`
        Count relevance to compute
    method : `str`, optional
        Metric implementation method, takes the values\
             `standard` and `industry`

    Returns
    -------
    score : `float`
        Metric score
    """

    score = np.mean(list(map(lambda x: (normalized_dcg(x, k, method)), list_relevances)))
    return score

list_relevances = [
        [0.99, 0.94, 0.88, 0.89, 0.72, 0.65],
        [0.99, 0.92, 0.93, 0.74, 0.61, 0.68], 
        [0.99, 0.96, 0.81, 0.73, 0.76, 0.69]
    ]  
k = 5
method = 'standard'
print(round(avg_ndcg(list_relevances, k, method), 5))

0.99958
